In [1]:
#importing packages
import pandas as pd
import json
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
#loading product data files
prods_1 = pd.read_csv('prods_1_of_2.csv', low_memory=False)
prods_2 = pd.read_csv('prods_2_of_2.csv', low_memory=False)

In [3]:
#both df's have many nan values in 'url', which is the key to join the second dataframe
prods_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     600000 non-null  int64  
 1   _id            600000 non-null  object 
 2   albumRelease   333289 non-null  object 
 3   numTracks      333289 non-null  float64
 4   name           599986 non-null  object 
 5   dateModified   599998 non-null  object 
 6   @id            599998 non-null  object 
 7   keywords       599553 non-null  object 
 8   image          599914 non-null  object 
 9   comment        126694 non-null  object 
 10  byArtist       599998 non-null  object 
 11  track          333289 non-null  object 
 12  @type          599998 non-null  object 
 13  datePublished  599998 non-null  object 
 14  @context       600000 non-null  object 
 15  duration       266708 non-null  object 
 16  offers         266709 non-null  object 
 17  duration_secs  266708 non-nul

In [4]:
prods_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469972 entries, 0 to 469971
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     469972 non-null  int64  
 1   _id            469972 non-null  object 
 2   @type          469972 non-null  object 
 3   dateModified   469972 non-null  object 
 4   track          230640 non-null  object 
 5   datePublished  469972 non-null  object 
 6   @context       469972 non-null  object 
 7   name           469962 non-null  object 
 8   albumRelease   230640 non-null  object 
 9   numTracks      230640 non-null  float64
 10  keywords       469674 non-null  object 
 11  @id            469972 non-null  object 
 12  byArtist       469972 non-null  object 
 13  image          469867 non-null  object 
 14  description    198333 non-null  object 
 15  duration       239332 non-null  object 
 16  url            239332 non-null  object 
 17  offers         239332 non-nul

In [5]:
#_id and url have the same values, but many url values = Nan
print(prods_1['url'].isna().sum())
print(prods_1['_id'].isna().sum())

333291
0


In [6]:
#seperate values of _id into a list, replace the 'https:' with '' so that the values are the same in the second data frame. Append the new list to the dataframe. 
url_list_1 = prods_1['_id'].values.tolist()

url_list_1_2 = []
for x in range(0, len(url_list_1)):
    url = url_list_1[x]
    url = url.replace("https:", "")
    url_list_1_2.append(url)

prods_1['url_2'] = url_list_1_2

In [7]:
url_list_2 = prods_2['_id'].values.tolist()

url_list_2_2 = []
for x in range(0, len(url_list_2)):
    url = url_list_2[x]
    url = url.replace("https:", "")
    url_list_2_2.append(url)

prods_2['url_2'] = url_list_2_2

In [8]:
#concatenate the 2 separate sets into 1 
prods = pd.concat([prods_1, prods_2], ignore_index=True)

In [9]:
#save to file
prods.to_csv('prods.csv')

In [10]:
#loading the second data set
#loading sales file
sales = pd.read_csv('1000000-bandcamp-sales.csv', index_col = 'Unnamed: 0')

#filter physical products out
df_dig = sales[(sales['item_type'] == 'a') | (sales['item_type'] == 't')]

In [11]:
#renaming the key columns, so that the dataframes can be merged
df_dig = df_dig.rename(columns={"url": "join_key"})
prods = prods.rename(columns={'url_2':'join_key'})

In [12]:
#merging the dataframes on left join. Left dataset is the sales dataset, the product information (right dataset) will be joined on the products sold
df = pd.merge(df_dig, prods, how='left', on='join_key')

In [13]:
#save dataframe
df.to_csv('complete_set.csv')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752599 entries, 0 to 752598
Data columns (total 47 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   _id_x                  752599 non-null  object 
 1   art_url                752599 non-null  object 
 2   item_type              752599 non-null  object 
 3   utc_date               752599 non-null  float64
 4   country_code           752599 non-null  object 
 5   track_album_slug_text  2237 non-null    object 
 6   country                752599 non-null  object 
 7   slug_type              752599 non-null  object 
 8   amount_paid_fmt        752599 non-null  object 
 9   item_price             752599 non-null  float64
 10  item_description       752593 non-null  object 
 11  art_id                 752459 non-null  float64
 12  join_key               752599 non-null  object 
 13  amount_paid            752599 non-null  float64
 14  releases               0 non-null   